Prepare state space trajectories for a lexical analysis.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
from collections import Counter, defaultdict
import itertools
from pathlib import Path
import pickle
from typing import Any

import datasets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split, cross_val_score
import torch
import transformers

from src.models import get_best_checkpoint
from src.analysis.state_space import StateSpaceAnalysisSpec
from src.models.integrator import ContrastiveEmbeddingModel, load_or_compute_embeddings

In [ ]:
# model_dir = "out/ce_model_phoneme_within_word_prefix_6_32"
model_dir = "out/ce_model_random_32"

# use a word-level equivalence dataset regardless of model, so that we can look up cohort facts
equiv_dataset_path = "data/timit_equiv_phoneme_6_1.pkl"
timit_corpus_path = "data/timit_phonemes"

phoneme_response_window = (0, 3)

In [ ]:
model = ContrastiveEmbeddingModel.from_pretrained(get_best_checkpoint(model_dir))
model.eval()

In [ ]:
with open(equiv_dataset_path, "rb") as f:
    equiv_dataset = pickle.load(f)

In [ ]:
timit_corpus = datasets.load_from_disk(timit_corpus_path)

In [ ]:
all_phonemes = set([phone["phone"] for words in timit_corpus["train"]["word_phonemic_detail"]
 for word in words
 for phone in word])

In [ ]:
cmudict_features = {
    "AA": "low back syllabic sonorant",
    "AE": "low front syllabic sonorant",
    "AH": "syllabic sonorant",
    "AO": "low back syllabic sonorant",
    "AW": "syllabic sonorant",
    "AY": "syllabic sonorant",
    "B": "anterior bilabial obstruent consonantal plosive voiced",
    "CH": "velar unvoiced",
    "D": "anterior coronal alveolar obstruent consonantal plosive voiced",
    "DH": "anterior alveolar obstruent consonantal fricative voiced",
    "EH": "low front syllabic sonorant",
    "ER": "sonorant",
    "EY": "syllabic sonorant",
    "F": "anterior labiodental obstruent consonantal fricative unvoiced",
    "G": "dorsal velar obstruent consonantal plosive voiced",
    "HH": "consonantal fricative unvoiced",
    "IH": "high front syllabic sonorant",
    "IY": "high front syllabic sonorant",
    "JH": "velar voiced",
    "K": "dorsal velar obstruent consonantal plosive unvoiced",
    "L": "anterior coronal alveolar approximant consonantal sonorant",
    "M": "anterior bilabial consonantal sonorant nasal",
    "N": "anterior coronal alveolar consonantal sonorant nasal",
    "NG": "dorsal consonantal sonorant nasal",
    "OW": "syllabic sonorant",
    "OY": "syllabic sonorant",
    "P": "anterior bilabial obstruent consonantal plosive unvoiced",
    "R": "anterior coronal alveolar approximant consonantal sonorant",
    "S": "anterior coronal alveolar obstruent consonantal sibilant fricative unvoiced",
    "SH": "coronal velar obstruent consonantal sibilant fricative unvoiced",
    "T": "anterior coronal alveolar obstruent consonantal plosive unvoiced",
    "TH": "anterior alveolar obstruent consonantal fricative unvoiced",
    "UH": "high back syllabic sonorant",
    "UW": "high back syllabic sonorant",
    "V": "anterior labiodental obstruent consonantal fricative voiced",
    "W": "approximant sonorant",  # y no place?
    "Y": "velar approximant sonorant",
    "Z": "anterior coronal alveolar obstruent consonantal sibilant fricative voiced",
    "ZH": "coronal alveolar obstruent consonantal sibilant fricative voiced",
}
cmudict_features = {k: v.split() for k, v in cmudict_features.items()}

In [ ]:
all_features = list(sorted(set(itertools.chain(*cmudict_features.values()))))
feature2idx = {f: i for i, f in enumerate(all_features)}

In [ ]:
cmudict_feature_idxs = {k: [feature2idx[f] for f in v] for k, v in cmudict_features.items()}

In [ ]:
feature_to_phonemes = {f: [k for k, v in cmudict_features.items() if f in v] for f in all_features}

In [ ]:
assert all(type(label) == str for label in equiv_dataset.class_labels), "Assumes dataset with phoneme labels"

In [ ]:
model_representations = load_or_compute_embeddings(model, equiv_dataset, model_dir, equiv_dataset_path)

In [ ]:
phoneme2idx = {p: i for i, p in enumerate(cmudict_features.keys())}
feature_matrix = np.zeros((len(feature2idx), len(phoneme2idx)), dtype=int)
for feature, phonemes in feature_to_phonemes.items():
    for phoneme in phonemes:
        feature_matrix[feature2idx[feature], phoneme2idx[phoneme]] = 1
feature_df = pd.DataFrame(feature_matrix, index=all_features, columns=list(cmudict_features.keys()))
sns.heatmap(feature_df)

## Extract representations

In [ ]:
equiv_frames_by_item = equiv_dataset.hidden_state_dataset.frames_by_item

In [ ]:
mean_rep = np.mean(model_representations, axis=0, keepdims=True)
std_rep = np.std(model_representations, axis=0, keepdims=True)

In [ ]:
phoneme_responses = defaultdict(list)
phoneme_agg_fn = np.mean
zscore = True

def get_phoneme_responses(item, idx):
    start_frame, end_frame = equiv_frames_by_item[idx]
    compression_ratio = (end_frame - start_frame) / len(item["input_values"])

    window_left, window_right = phoneme_response_window

    for word in item["word_phonemic_detail"]:
        for phone in word:
            phone_start = start_frame + int(phone["start"] * compression_ratio)
            phone_end = start_frame + int(phone["stop"] * compression_ratio)

            response = model_representations[phone_end + window_left:phone_end + window_right]

            if zscore:
                response = (response - mean_rep) / std_rep

            phoneme_responses[phone["phone"]].append(phoneme_agg_fn(response, axis=0))

timit_corpus.map(get_phoneme_responses, with_indices=True)

## Aggregate by feature

In [ ]:
feature_responses = defaultdict(list)
for feature, phonemes in feature_to_phonemes.items():
    for phoneme in phonemes:
        feature_responses[feature].extend(phoneme_responses[phoneme])

feature_responses = {k: np.stack(v) for k, v in feature_responses.items()}

## Save

In [ ]:
with open(Path(output_dir) / "phoneme_responses.pkl", "wb") as f:
    pickle.dump(phoneme_responses, f)
with open(Path(output_dir) / "feature_responses.pkl", "wb") as f:
    pickle.dump(feature_responses, f)

## Feature selectivity

In [ ]:
# num_features * num_dimensions
feature_responses_mat = np.array([feature_responses_i.mean(axis=0)
                                  for feature_responses_i in feature_responses.values()])

In [ ]:
# For each feature--hidden unit pair, calculate feature selectivity index:
# FSI_ij receives 1 point for every feature to which hidden unit i responds
# more weakly than it did to feature j by a threshold (0.15).
feature_selectivity_threshold = 0.4
feature_selectivity = np.zeros_like(feature_responses_mat)

for hidden_idx in range(feature_selectivity.shape[1]):
    for feature_idx in range(feature_selectivity.shape[0]):
        feature_response = feature_responses_mat[feature_idx, hidden_idx]

        other_feature_responses = np.concatenate([
            feature_responses_mat[:feature_idx, hidden_idx],
            feature_responses_mat[feature_idx + 1:, hidden_idx],
        ])
        feature_selectivity[feature_idx, hidden_idx] = (np.abs(feature_response - other_feature_responses) > feature_selectivity_threshold).sum()

In [ ]:
sns.clustermap(feature_selectivity, yticklabels=feature_to_phonemes.keys(), xticklabels=False)